In [129]:
%pip install -qU langchain-ollama
%pip install load_dotenv
%pip install langchain-ollama
%pip install langchain_community
%pip install langchain
%pip install weaviate-client
%pip install langchain-openai
%pip install langchain-weaviate
%pip install fastapi



Note: you may need to restart the kernel to use updated packages.


c:\Users\khale\machine_learning\SMS\weaviate\venv\Lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedWriter name=3>
  return process_handler(cmd, _system_body)
c:\Users\khale\machine_learning\SMS\weaviate\venv\Lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=4>
  return process_handler(cmd, _system_body)
c:\Users\khale\machine_learning\SMS\weaviate\venv\Lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  return process_handler(cmd, _system_body)


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [130]:
%pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [131]:
from dotenv import load_dotenv
import os

In [132]:
from langchain_ollama import OllamaEmbeddings
import os
from langchain.embeddings.openai import OpenAIEmbeddings
import weaviate
from weaviate.classes.init import Auth

embeddings_openai = OpenAIEmbeddings(openai_api_key = os.getenv("OPENAI_API_KEY"))
embeddings_llama = OllamaEmbeddings(
    model="mxbai-embed-large",
)

# Best practice: store your credentials in environment variables
wcd_url = os.environ["WEAVAITE_URL"]
wcd_api_key = os.environ["WEAVAITE_API_KEY"]
openai_api_key = os.environ["OPENAI_API_KEY"]
cohere_api_key = os.environ["COHERE_API_KEY"]

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=wcd_url,  # Replace with your Weaviate Cloud URL
    auth_credentials=Auth.api_key(wcd_api_key),  # Replace with your Weaviate Cloud key
    headers={'X-OpenAI-Api-key': openai_api_key, "X-Cohere-Api-Key": cohere_api_key}  # Replace with your OpenAI API key
)

KeyError: 'COHERE_API_KEY'

In [ ]:
import weaviate.classes as wvc

try:
    questions = client.collections.create(
        name="Question",
        vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_openai(),    # Set the vectorizer to "text2vec-openai" to use the OpenAI API for vector-related operations
        generative_config=wvc.config.Configure.Generative.cohere(),             # Set the generative module to "generative-cohere" to use the Cohere API for RAG
        properties=[
            wvc.config.Property(
                name="question",
                data_type=wvc.config.DataType.TEXT,
            ),
            wvc.config.Property(
                name="answer",
                data_type=wvc.config.DataType.TEXT,
            ),
            wvc.config.Property(
                name="category",
                data_type=wvc.config.DataType.TEXT,
            )
        ]
    )

    print(questions.config.get(simple=False))

except weaviate.RequestError as e:
    print(e)

_CollectionConfig(name='Question', description=None, generative_config=_GenerativeConfig(generative=<GenerativeSearches.COHERE: 'generative-cohere'>, model={}), inverted_index_config=_InvertedIndexConfig(bm25=_BM25Config(b=0.75, k1=1.2), cleanup_interval_seconds=60, index_null_state=False, index_property_length=False, index_timestamps=False, stopwords=_StopwordsConfig(preset=<StopwordsPreset.EN: 'en'>, additions=None, removals=None)), multi_tenancy_config=_MultiTenancyConfig(enabled=False, auto_tenant_creation=False, auto_tenant_activation=False), properties=[_Property(name='question', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WORD: 'word'>, vectorizer_config=_PropertyVectorizerConfig(skip=False, vectorize_property_name=True), vectorizer='text2vec-openai'), _Property(name='answer', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=Tr

In [ ]:
# define input structure
client.schema.delete_all()
client.schema.get()
schema = {
    "classes": [
        {
            "class": "Chatbot",
            "description": "Documents for chatbot",
            "vectorizer": "text2vec-openai",
            "moduleConfig": {"text2vec-openai": {"model": "ada", "type": "text"}},
            "properties": [
                {
                    "dataType": ["text"],
                    "description": "The content of the paragraph",
                    "moduleConfig": {
                        "text2vec-openai": {
                            "skip": False,
                            "vectorizePropertyName": False,
                        }
                    },
                    "name": "content",
                },
            ],
        },
    ]
}

client.schema.create(schema)

vectorstore = Weaviate(client, "Chatbot", "content", attributes=["source"])

In [ ]:
from langchain_weaviate.vectorstores import WeaviateVectorStore

In [ ]:
import os
print(os.listdir('.'))

['.env', '.gitignore', 'main.ipynb', 'main.py', 'requirements.txt', 'state_of_the_union.txt', 'Syrian_revolution.pdf', 'True_Multimodal_RAG.ipynb', 'venv', 'weaviate_LangChain.ipynb']


In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

In [ ]:
from langchain_community.document_loaders import TextLoader

# Try different encodings
encodings_to_try = ['utf-8', 'latin-1', 'ascii', 'utf-16']

for encoding in encodings_to_try:
    try:
        raw_documents = TextLoader(
            r'C:\Users\khale\machine_learning\SMS\weaviate\state_of_the_union.txt', 
            encoding=encoding
        ).load()
        print(f"Successfully loaded with {encoding} encoding.")
        break
    except UnicodeDecodeError:
        print(f"Failed to load with {encoding} encoding.")
    except Exception as e:
        print(f"An unexpected error occurred with {encoding} encoding: {str(e)}")
else:
    print("Failed to load the file with any of the attempted encodings.")

Successfully loaded with utf-8 encoding.


In [ ]:
raw_documents = TextLoader(
            r'C:\Users\khale\machine_learning\SMS\weaviate\state_of_the_union.txt', 
            encoding=encoding
        ).load()

In [ ]:

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(raw_documents)

In [ ]:
print(documents)

[Document(metadata={'source': 'C:\\Users\\khale\\machine_learning\\SMS\\weaviate\\state_of_the_union.txt'}, page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian

In [ ]:
from langchain_weaviate.vectorstores import WeaviateVectorStore



In [ ]:
db = WeaviateVectorStore.from_documents(documents, embedding= embeddings_openai, client=client)

In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)

# Print the first 100 characters of each result
for i, doc in enumerate(docs):
    print(f"\nDocument {i+1}:")
    print(doc.page_content[:100] + "...")


Document 1:
Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Ac...

Document 2:
A former top litigator in private practice. A former federal public defender. And from a family of p...

Document 3:
Vice President Harris and I ran for office with a new economic vision for America. 

Invest in Ameri...

Document 4:
And for our LGBTQ+ Americans, let’s finally get the bipartisan Equality Act to my desk. The onslaugh...


In [ ]:
from weaviate.classes.query import Filter

for filter_str in ["blah.txt", "state_of_the_union.txt"]:
    search_filter = Filter.by_property("source").equal(filter_str)
    filtered_search_results = db.similarity_search(query, filters=search_filter)
    print(len(filtered_search_results))
    if filter_str == "state_of_the_union.txt":
        assert len(filtered_search_results) > 0  # There should be at least one result
    else:
        assert len(filtered_search_results) == 0  # There should be no results

0
4


In [ ]:
search_filter = Filter.by_property("source").equal("state_of_the_union.txt")
filtered_search_results = db.similarity_search(query, filters=search_filter, k=3)
assert len(filtered_search_results) <= 3

In [ ]:
vectorstore = Weaviate(
    client=client,
    embedding=embeddings_openai,
    by_text=False  # Since we're providing embeddings
)

TypeError: Weaviate.__init__() missing 2 required positional arguments: 'index_name' and 'text_key'

In [ ]:
from langchain.vectorstores import Weaviate

# Initialize LangChain Weaviate vector store
vectorstore = Weaviate(
    client=client,
    embedding=embeddings_openai,
    by_text=False  # Since we're providing embeddings
)


TypeError: Weaviate.__init__() missing 2 required positional arguments: 'index_name' and 'text_key'

WEAVIATE YOUTUBE CHANNEL

In [153]:
import weaviate
import weaviate.classes.config as wc


reviews = client.collections.create(
    name="Review",
    properties=[
        wc.Property(name="username", data_type=wc.DataType.TEXT, skip_vectorization=True ),
        wc.Property(name="content", data_type=wc.DataType.TEXT ),
        wc.Property(name="tmdb_id", data_type=wc.DataType.TEXT, skip_vectorization=True ),
    ],
    vectorizer_config=wc.Configure.Vectorizer.text2vec_cohere(),
    generative_config=wc.Configure.Generative.openai(
        model="gpt-4o",
        max_tokens=128000
    ),
    inverted_index_config=wc.Configure.inverted_index(
        index_property_length=True,
    ),
)

movies = client.collections.create(
    name="Movie",
    properties=[
        wc.Property(name="title", data_type=wc.DataType.TEXT ),
        wc.Property(name="tagline", data_type=wc.DataType.TEXT ),
        wc.Property(name="overview", data_type=wc.DataType.TEXT ),
        wc.Property(name="runtime", data_type=wc.DataType.INT ),
        wc.Property(name="release_date", data_type=wc.DataType.DATE ),
        wc.Property(name="description", data_type=wc.DataType.TEXT ),
        wc.Property(name="tmdb_id", data_type=wc.DataType.TEXT, skip_vectorization=True ),
    ],
    vectorizer_config=wc.Configure.Vectorizer.text2vec_cohere(),
    generative_config=wc.Configure.Generative.openai(
        model="gpt-4o",
        max_tokens=128000
    ),
    vector_index_config=wc.Configure.VectorIndex.hnsw(
        distance_metric=wc.VectorDistances.COSINE,
    ),
    references=[
        wc.ReferenceProperty(
            name="hasReview",
            target_collection="Review",
        )
    ]
)

print(reviews.config.get(simple=False))


UnexpectedStatusCodeError: Collection may not have been created properly.! Unexpected status code: 422, with response body: {'error': [{'message': 'class name Review already exists'}]}.

In [ ]:
try:
    reviews = client.collections.get("Item")
    print(reviews.config.get(simple=False))
except Exception as e:
    print("Item collection does not exist")


Item collection does not exist


In [154]:
def create_weaviate_schema():
    # Check if 'Item' class exists
    try:
        items =client.collections.get("Item")
        print(items.config.get(simple=False))
    except Exception as e:
        print("Item collection already exists: Creating new Item collection")
        client.collections.create(
            name="Item",
            properties=[
                wc.Property(name="item_id", data_type=wc.DataType.TEXT, skip_vectorization=True),
                wc.Property(name="name", data_type=wc.DataType.TEXT),
                wc.Property(name="description", data_type=wc.DataType.TEXT, optional=True),
                wc.Property(name="box", data_type=wc.DataType.TEXT, skip_vectorization=True),
                wc.Property(name="workspace", data_type=wc.DataType.TEXT, skip_vectorization=True),
            ],
            vectorizer_config=wc.Configure.Vectorizer.text2vec_openai(),
            generative_config=wc.Configure.Generative.openai(
        model="gpt-4o",
        max_tokens=128000
    ),
            vector_index_config=wc.Configure.VectorIndex.hnsw(
                distance_metric=wc.VectorDistances.COSINE,
            ),
        )

create_weaviate_schema()

Item collection already exists: Creating new Item collection


In [155]:
try:
    items = client.collections.get("Item")
    print(items.config.get(simple=False))
except Exception as e:
    print("Item collection does not exist")

_CollectionConfig(name='Item', description=None, generative_config=_GenerativeConfig(generative=<GenerativeSearches.OPENAI: 'generative-openai'>, model={'maxTokensProperty': 128000, 'model': 'gpt-4o'}), inverted_index_config=_InvertedIndexConfig(bm25=_BM25Config(b=0.75, k1=1.2), cleanup_interval_seconds=60, index_null_state=False, index_property_length=False, index_timestamps=False, stopwords=_StopwordsConfig(preset=<StopwordsPreset.EN: 'en'>, additions=None, removals=None)), multi_tenancy_config=_MultiTenancyConfig(enabled=False, auto_tenant_creation=False, auto_tenant_activation=False), properties=[_Property(name='item_id', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WORD: 'word'>, vectorizer_config=_PropertyVectorizerConfig(skip=True, vectorize_property_name=True), vectorizer='text2vec-openai'), _Property(name='name', description=None, data_type=<DataT

In [156]:
# Create the "Manual" collection with a "manual_id" property
def create_weaviate_manuals():
    try:
        manuals = client.collections.get("Manual")
        print(manuals.config.get(simple=False))
    except Exception as e:
        manuals = client.collections.create(
            name="Manual",
            properties=[
                wc.Property(name="manual_id", data_type=wc.DataType.TEXT, skip_vectorization=True),
                wc.Property(name="content", data_type=wc.DataType.TEXT),
                wc.Property(name="type", data_type=wc.DataType.TEXT),
                wc.Property(name="workspace", data_type=wc.DataType.TEXT, skip_vectorization=True),
            ],
            vectorizer_config=wc.Configure.Vectorizer.text2vec_openai(),
            generative_config=wc.Configure.Generative.openai(
        model="gpt-4o",
        max_tokens=128000
    ),
            vector_index_config=wc.Configure.VectorIndex.hnsw(
                distance_metric=wc.VectorDistances.COSINE,
            ),
        )
        print(manuals.config.get(simple=False))

create_weaviate_manuals()


_CollectionConfig(name='Manual', description=None, generative_config=_GenerativeConfig(generative=<GenerativeSearches.OPENAI: 'generative-openai'>, model={'maxTokensProperty': 128000, 'model': 'gpt-4o'}), inverted_index_config=_InvertedIndexConfig(bm25=_BM25Config(b=0.75, k1=1.2), cleanup_interval_seconds=60, index_null_state=False, index_property_length=False, index_timestamps=False, stopwords=_StopwordsConfig(preset=<StopwordsPreset.EN: 'en'>, additions=None, removals=None)), multi_tenancy_config=_MultiTenancyConfig(enabled=False, auto_tenant_creation=False, auto_tenant_activation=False), properties=[_Property(name='manual_id', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WORD: 'word'>, vectorizer_config=_PropertyVectorizerConfig(skip=True, vectorize_property_name=True), vectorizer='text2vec-openai'), _Property(name='content', description=None, data_type

In [ ]:
items = client.collections.get("Item")
manuals =client.collections.get("Manual")

In [157]:
def create_item(items,item_id, name, description, box, workspace):
    items.data.insert(
        properties={
            "item_id": item_id,
            "name": name,
            "description": description,
            "box": box,
            "workspace": workspace,
        },
    )

create_item(items = items,item_id="item-12345",name="Rice Cooker",description="A high-quality rice cooker from Hitachi",box="Cabinet2",workspace="Kitchen")


In [ ]:
def update_item(items,item_id, name, description, box, workspace):
    items.data.update(
        properties={
            "item_id": item_id,
            "name": name,
            "description": description,
            "box": box,
            "workspace": workspace,
        },
    )

update_item(items = items,item_id="item-12345",name="rice cooker",description="A high-quality rice cooker from Hitachi.",box="Cabinet2",workspace="Kitchen")


TypeError: _DataCollectionAsync.update() missing 1 required positional argument: 'uuid'

In [ ]:
def delete_item(items, uuid):
    delted_item = items.data.delete_by_id(uuid)
    print(delted_item)
delete_item(items, "77405086-8c27-4e7c-afdc-c0950832a89e")

True


In [ ]:
# Update an item
where_filter = {
    "path": ["item_id"],
    "operator": "Equal",
    "valueText": "item-12345",
}
# Assuming you have a function to find the object's UUID based on the filter
item_uuid = find_object_uuid("Item", where_filter)
client.data_object.update(
    data_object={
        "description": "A high-quality rice cooker.",
    },
    class_name="Item",
    uuid=item_uuid,
)

In [ ]:
# Delete an item
client.data_object.delete(
    class_name="Item",
    uuid=item_uuid,
)

In [158]:
#Keyword Search
response = items.query.bm25(
    query="rice",
    limit=2
)
print(response)
# Assuming 'response' is an instance of QueryReturn as returned from Weaviate
objects = response.objects  # Access the 'objects' attribute of the response
print(objects)
for obj in objects:
    # Accessing UUID
    uuid = obj.uuid
    print("-------------")
    print("UUID:", uuid)

    # Accessing Metadata (if needed)
    # creation_time = obj.metadata.creation_time
    # print("Creation Time:", creation_time)

    # Accessing properties
    properties = obj.properties
    # workspace = properties['workspace']
    # description = properties['description']
    # box = properties['box']
    item_id = properties['item_id']
    name = properties['name']
    description = properties['description']
    # print("Workspace:", workspace)
    # print("Description:", description)
    # print("Box:", box)
    print("Item ID:", item_id)
    print("Name:", name)
    print("Description:", description)
    # Accessing collection if needed
    collection = obj.collection
    print("Collection:", collection)



QueryReturn(objects=[Object(uuid=_WeaviateUUIDInt('d9e21364-9714-4804-9a5a-a08ecb7dde37'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None), properties={'workspace': 'Kitchen', 'description': 'A high-quality rice cooker from Hitachi', 'box': 'Cabinet2', 'item_id': 'item-12345', 'name': 'Rice Cooker'}, references=None, vector={}, collection='Item')])
[Object(uuid=_WeaviateUUIDInt('d9e21364-9714-4804-9a5a-a08ecb7dde37'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None), properties={'workspace': 'Kitchen', 'description': 'A high-quality rice cooker from Hitachi', 'box': 'Cabinet2', 'item_id': 'item-12345', 'name': 'Rice Cooker'}, references=None, vector={}, collection='Item')]
-------------
UUID: d9e21364-9714-4804-9a5a-a08ecb7dde37
Item ID: item-12345


In [161]:
#For generative search and llm response
response = items.generate.near_text(
    query="something to cook with",
    limit=5,
    # filters=wq.Filter.by_property("runtime").less_than(100) & wq.Filter.by_property("runtime").greater_than(50),
    # prompt for every single object
    # single_prompt="How can I use this item?: {description}",
    grouped_task="where should I put these items?: {description}, please answer shortly"

)

# the response for the grouped task will be in response.generated
print(response.generated)
#the return for the single prompt will be in object.generated
for o in response.objects:
    print("-------------")
    print("Name:", o.properties['name'])
    print("Description:", o.properties['description'])
    print(o.generated)



-------------
Name: Rice Cooker
Description: A high-quality rice cooker from Hitachi
None


In [162]:
print(response.generated)


Cabinet2 in the Kitchen.


In [ ]:
test = client.collections.create(
    name="test",
    properties=[
        wc.Property(name="name", data_type=wc.DataType.TEXT)
    ],
    vectorizer_config=wc.Configure.Vectorizer.text2vec_openai(),
    generative_config=wc.Configure.Generative.openai(),
    vector_index_config=wc.Configure.VectorIndex.hnsw(
                distance_metric=wc.VectorDistances.COSINE,
            ),
)

print(test.config.get(simple=False))


_CollectionConfig(name='Test', description=None, generative_config=_GenerativeConfig(generative=<GenerativeSearches.OPENAI: 'generative-openai'>, model={}), inverted_index_config=_InvertedIndexConfig(bm25=_BM25Config(b=0.75, k1=1.2), cleanup_interval_seconds=60, index_null_state=False, index_property_length=False, index_timestamps=False, stopwords=_StopwordsConfig(preset=<StopwordsPreset.EN: 'en'>, additions=None, removals=None)), multi_tenancy_config=_MultiTenancyConfig(enabled=False, auto_tenant_creation=False, auto_tenant_activation=False), properties=[_Property(name='name', description=None, data_type=<DataType.TEXT: 'text'>, index_filterable=True, index_range_filters=False, index_searchable=True, nested_properties=None, tokenization=<Tokenization.WORD: 'word'>, vectorizer_config=_PropertyVectorizerConfig(skip=False, vectorize_property_name=True), vectorizer='text2vec-openai')], references=[], replication_config=_ReplicationConfig(factor=1, async_enabled=False), reranker_config=Non

In [ ]:
inserted_data= test.data.insert(
    properties={
        "name": "test",
        "vector": ""
    }
)

print(inserted_data)



c218e9d9-ffd1-4a44-835e-1a03c64470de


In [ ]:
where_filter = {
    "path": ["name"],
    "operator": "Equal",
    "valueText": "test",
}
response = test.query.bm25(
    query="c218e9d9-ffd1-4a44-835e-1a03c64470de",
    limit=1
)
objects = response.objects
if(len(objects) > 0):
    for obj in objects:
        print(obj.properties)
else:
    print("No objects found")


No objects found


In [ ]:
import weaviate.classes.query as wq
response = test.query.near_text( 
    query="c218e9d9-ffd1-4a44-835e-1a03c64470de",
    limit=1,
    # filters=wq.Filter.by_id().equal("c218e9d9-ffd1-4a44-835e-1a03c64470de")
)
objects = response.objects
if(len(objects) > 0):
    for obj in objects:
        print(obj.properties)
else:
    print("No objects found")

{'name': 'test'}
